In [46]:

# coding: utf-8

# In[3]:


import pandas as pd
import numpy as np
#import matplotlib as mpl
#mpl.use('Agg')
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import KBinsDiscretizer
from helpers import season_splitter
from helpers import plot_ys
from helpers import split_hs_test
pd.set_option('display.max_columns', 100)



In [53]:
# In[ ]:

INPUT_FOLDER = ''
OUTPUT_FOLDER = ''

# # Load and preprocess the data

# In[65]:

filetxt = open(OUTPUT_FOLDER + "FS_RF_seasons.txt", "w")
foridge = open(OUTPUT_FOLDER + "FS_RF_seasons_for_regression.txt", "w")

#for i,season in enumerate(["Spring","Summer","Autumn","Winter"]):
#for i,season in enumerate(["Spring"]):
# In[66]:
tot_df=pd.read_csv(INPUT_FOLDER + 'regression_mat_year.csv',index_col=0)
tot_df = season_splitter(tot_df)
tot_df = tot_df[1]
# create columns with coordinate velocities output
tot_df['u_x']=tot_df['u']*np.cos(np.radians(tot_df['direction']))
tot_df['u_y']=tot_df['u']*np.sin(np.radians(tot_df['direction']))
# create columns with coordinate velocities input top mast anemometer
tot_df['u_top_x']=tot_df['u_top']*np.cos(np.radians(tot_df['direction_top']))
tot_df['u_top_y']=tot_df['u_top']*np.sin(np.radians(tot_df['direction_top']))
# drop the columns which are not used anymore
tot_df=tot_df.drop(columns=['u', 'u_top', 'direction', 'direction_top'])
tot_df=tot_df.iloc[0:,:]


# # Random forest feature selection
# Pipeline: Discretize the output -> Random forest
# <br>Output: u_x, u_y, z

# ## Prepare the input and output

# In[12]:


x = np.array(tot_df.drop(columns=['u_x', 'u_y','u_z']))
y_continue = np.array(tot_df[['u_x', 'u_y']])


# ## Discretize the output

# In[13]:


discretizer = KBinsDiscretizer(n_bins=20, encode='ordinal', strategy='uniform')
discretizer.fit(y_continue)
y_disc = discretizer.transform(y_continue)


# ## Split train and test

# In[14]:


x_tr, x_te, y_tr, y_te = train_test_split(x, y_disc, test_size=0.2, random_state=50)
x_tr, x_ev, y_tr, y_ev = train_test_split(x_tr, y_tr, test_size=0.6, random_state=50)

In [67]:
print y_te
print y

[[10.  7.]
 [ 7.  7.]
 [ 8.  9.]
 ...
 [ 9.  8.]
 [12.  7.]
 [ 9.  6.]]


NameError: name 'y' is not defined

In [54]:
 # ## Random forest

    # In[15]:
    
    #y_tr_cont = discretizer.inverse_transform(y_tr)


    # In[16]:


rf = RandomForestClassifier(n_estimators=200, max_depth=None, criterion='gini', random_state=0)
rf.fit(x_tr, y_tr)
print("Random forest for one season finished")

# Prediction
#y_pred = rf.predict(x_te)

# Transfer back to the original data
#y_pred = discretizer.inverse_transform(y_pred)
#y_te = discretizer.inverse_transform(y_te)


Random forest for one season finished


In [65]:
x_te_hs, y_te_hs_ = split_hs_test(x_te,y_te)
y_pred_hs=[]
y_te_hs=[]
mse = []
for hs in x_te_hs:
    ans=rf.predict(hs)
    #ans=discretizer.inverse_transform(ans)
    y_pred_hs.append(ans)
for hs in y_te_hs_:
    #ans=discretizer.inverse_transform(hs)
    y_te_hs.append(ans)
for i in range(len(y_te_hs)):
    ans = np.mean(np.square(y_te_hs[i]-y_pred_hs[i]))
    mse.append(ans)

ValueError: operands could not be broadcast together with shapes (5341,2) (5464,2) 

In [66]:
print mse
print y_pred_hs[1]
print y_continue


[]
[[9. 8.]
 [9. 8.]
 [9. 7.]
 ...
 [9. 8.]
 [8. 7.]
 [9. 8.]]
[[-0.37087667  0.47688029]
 [-0.54953379  0.63073709]
 [-0.45609205  0.88924382]
 ...
 [-0.91082134  0.01616873]
 [ 0.42543056 -1.34194208]
 [-0.35319209 -1.65395052]]


In [63]:
# Plot the prediction result
plot_ys(y_pred_hs,y_te_hs,OUTPUT_FOLDER,save=True,interval=[0,500],name='graph')

In [ ]:
# ## Print the result(feature importance)

    # In[18]:


feat_labels = tot_df.drop(columns=['u_x', 'u_y','u_z']).columns


# In[74]:
importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]
important_features = []
importance_accum = 0
#open("feature_importance.txt", 'w').close
filetxt.write("\n For the %s: \n" % season)
for f in range(x_tr.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 50, feat_labels[indices[f]], importances[indices[f]]))
    filetxt.write("%2d) %-*s %f \n" % (f + 1, 50, feat_labels[indices[f]], importances[indices[f]]))
    if importance_accum < 0.80:
        importance_accum = importance_accum + importances[indices[f]]
        important_features.append(feat_labels[indices[f]])
plt.figure()
plt.title("Feature importances for season %s" % season)
plt.bar(range(x_tr.shape[1]), importances, color="b", align="center")
plt.xticks(range(x_tr.shape[1]), feat_labels, fontsize=5, rotation=90)
plt.subplots_adjust(bottom=0.20)
plt.grid()
plt.savefig(OUTPUT_FOLDER + 'The_%s.eps' % season)
filetxt.write("\n The top 80% important features are: \n")
for i in range(len(important_features)):
    filetxt.write("%s \n" % important_features[i])
filetxt.write("%i features on %i" % (len(important_features), x_tr.shape[1]))
foridge.write("%s\n" % season)
for i in range(len(important_features)):
    foridge.write("%s\n" % important_features[i])
filetxt.close()
foridge.close()